In [1]:
#settings 
from settings.settings import data_settings

#imports
import pickle
import pandas as pd 
import numpy as np 
from datetime import datetime
import matplotlib as mpl 
import matplotlib.pyplot as plt

pd.options.display.max_columns = 500
pd.options.display.max_seq_items = 500
pd.options.display.max_rows = 500

In [3]:
# Loading data and making it ready
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv"
tp_to_sl = 1
slatr = 5
tp_to_sl_s = [1,1.25,1.5,1.75,2,4,6]
date_time_format = "%Y-%m-%d %H:%M:%S"
name = "SP500m_H1_202009231400_202207152200"

df = pd.read_csv(f"{csv_path}/preprocessed/{name}.csv")

NameError: name 'pd' is not defined

In [4]:
def print_nans(df):
    for column in df.columns:
        for j in range(df.shape[0]):
            if(pd.isna(df[column].iloc[j])):
                print( column , ":",j)

def print_infs(df):
    for column in df.columns:
        for j in range(df.shape[0]):
            if(df[column].iloc[j]==np.inf):
                print( column , ":",j)
                

print("Number of rows with na values:",df.shape[0] - df.dropna().shape[0])

Number of rows with na values: 0


In [5]:
split_year = 2022
split_month = 1
split_day = 1
date_time_format = "%Y-%m-%d %H:%M:%S"

df["DateTime"] = pd.to_datetime(df.pop("DateTime"), format=date_time_format)

train_df = df.loc[df.DateTime < datetime(
    split_year, split_month, split_day)].reset_index(drop=True)
test_df = df.loc[df.DateTime >= datetime(
    split_year, split_month, split_day)].reset_index(drop=True)


In [7]:
buy_features = [
# 'Open',
# 'High',
# 'Low',
# 'Close',
'TickVol',
'Spread',
'PPO',
'HIST',
'PPO_SIGNAL',
'ROC',
'RSI',
'STOCHRSI',
'STOCHRSI_D',
'STOCHRSI_K',
'TSI',
# 'BOLLINGER_HBAND',
'BOLLINGER_HBAND_INDICATOR',
# 'BOLLINGER_LBAND',
'BOLLINGER_LBAND_INDICATOR',
# 'BOLLINGER_MAVG',
'BOLLINGER_PBAND',
'BOLLINGER_WBAND',
'ULCER_INDEX',
'DPO',
'KST',
'KST_SIG',
'MACD',
'MACD_DIFF',
'MACD_SIGNAL',
'STC',
'TRIX',
'CUMULATIVE_RETURN',
'DAILY_LOG_RETURN',
'DAILY_RETURN',
'PVO',
'PVO_HIST',
'PVO_SIGNAL',
'CCI',
'STOCH',
'STOCH_SIGNAL',
'ULTIMATE_OSCILLATOR',
'WILLIAMS_R',
'AVERAGE_TRUE_RANGE',
# 'DONCHIAN_CHANNEL_HBAND',
# 'DONCHIAN_CHANNEL_LBAND',
# 'DONCHIAN_CHANNEL_MBAND',
'DONCHIAN_CHANNEL_PBAND',
'DONCHIAN_CHANNEL_WBAND',
# 'KELTNER_CHANNEL_HBAND',
'KELTNER_CHANNEL_HBAND_INDICATOR',
# 'KELTNER_CHANNEL_LBAND',
'KELTNER_CHANNEL_LBAND_INDICATOR',
# 'KELTNER_CHANNEL_MBAND',
'KELTNER_CHANNEL_PBAND',
'KELTNER_CHANNEL_WBAND',
'ADX',
'ADX_NEG',
'ADX_POS',
'PSAR_DOWN',
'PSAR_DOWN_INDICATOR',
'PSAR_UP',
'PSAR_UP_INDICATOR',
'VORTEX_INDICATOR_NEG',
'VORTEX_INDICATOR_POS',
'EASE_OF_MOVEMENT',
'SMA_EASE_OF_MOVEMENT',
'ACC_DIST_INDEX',
'CHAIKIN_MONEY_FLOW',
'MONEY_FLOW_INDEX',
# 'VOLUME_WEIGHTED_AVERAGE_PRICE',
'FORCE_INDEX',
# 'NEGATIVE_VOLUME_INDEX',
'ON_BALANCE_VOLUME',
'VOLUME_PRICE_TREND',
'AWESOME_OSCILLATOR',
# 'ICHIMOKU_A',
# 'ICHIMOKU_B',
# 'ICHIMOKU_BASE_LINE',
# 'ICHIMOKU_CONVERSION_LINE',
'MASS_INDEX',
'AROON_DOWN',
'AROON_INDICATOR',
'AROON_UP',
# 'EMA_INDICATOR_012',
# 'EMA_INDICATOR_024',
# 'EMA_INDICATOR_048',
# 'EMA_INDICATOR_072',
# 'EMA_INDICATOR_096',
# 'EMA_INDICATOR_120',
# 'EMA_INDICATOR_144',
# 'EMA_INDICATOR_168',
# 'EMA_INDICATOR_192',
# 'SMA_INDICATOR_012',
# 'SMA_INDICATOR_024',
# 'SMA_INDICATOR_048',
# 'SMA_INDICATOR_072',
# 'SMA_INDICATOR_096',
# 'SMA_INDICATOR_120',
# 'SMA_INDICATOR_144',
# 'SMA_INDICATOR_168',
# 'SMA_INDICATOR_192',
'Day_sin',
'Day_cos',
'One_Year_sin',
'One_Year_cos',
'Two_Year_sin',
'Two_Year_cos',
'Three_Year_sin',
'Three_Year_cos',
'Four_Year_sin',
'Four_Year_cos',
'Five_Year_sin',
'Five_Year_cos',
# 'buy_1',
# 'buy_1.25',
# 'buy_1.5',
# 'buy_1.75',
# 'buy_2',
# 'buy_4',
# 'buy_6',
# 'sell_1',
# 'sell_1.25',
# 'sell_1.5',
# 'sell_1.75',
# 'sell_2',
# 'sell_4',
# 'sell_6',
'BOLLINGER_HBAND_C',
'BOLLINGER_LBAND_C',
'BOLLINGER_MAVG_C',
'DONCHIAN_CHANNEL_HBAND_C',
'DONCHIAN_CHANNEL_LBAND_C',
'DONCHIAN_CHANNEL_MBAND_C',
'EMA_INDICATOR_012_C',
'EMA_INDICATOR_024_C',
'EMA_INDICATOR_048_C',
'EMA_INDICATOR_072_C',
'EMA_INDICATOR_096_C',
'EMA_INDICATOR_120_C',
'EMA_INDICATOR_144_C',
'EMA_INDICATOR_168_C',
'EMA_INDICATOR_192_C',
'SMA_INDICATOR_012_C',
'SMA_INDICATOR_024_C',
'SMA_INDICATOR_048_C',
'SMA_INDICATOR_072_C',
'SMA_INDICATOR_096_C',
'SMA_INDICATOR_120_C',
'SMA_INDICATOR_144_C',
'SMA_INDICATOR_168_C',
'SMA_INDICATOR_192_C',
# 'Open_C',
# 'High_C',
# 'Low_C',
'ICHIMOKU_A_C',
'ICHIMOKU_B_C',
'ICHIMOKU_BASE_LINE_C',
'ICHIMOKU_CONVERSION_LINE_C',
'KELTNER_CHANNEL_HBAND_C',
'KELTNER_CHANNEL_LBAND_C',
'KELTNER_CHANNEL_MBAND_C',
'NEGATIVE_VOLUME_INDEX_C',
'VOLUME_WEIGHTED_AVERAGE_PRICE_C',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_024',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_048',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_072',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_048',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_072',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_072',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_120_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_120_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_120_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_144_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_144_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_168_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_192',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_024',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_048',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_072',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_048',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_072',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_072',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_120_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_120_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_120_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_144_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_144_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_168_TO_SMA_INDICATOR_192',
# 'DateTime',
]

sell_features = [
# 'Open',
# 'High',
# 'Low',
# 'Close',
'TickVol',
'Spread',
'PPO',
'HIST',
'PPO_SIGNAL',
'ROC',
'RSI',
'STOCHRSI',
'STOCHRSI_D',
'STOCHRSI_K',
'TSI',
# 'BOLLINGER_HBAND',
'BOLLINGER_HBAND_INDICATOR',
# 'BOLLINGER_LBAND',
'BOLLINGER_LBAND_INDICATOR',
# 'BOLLINGER_MAVG',
'BOLLINGER_PBAND',
'BOLLINGER_WBAND',
'ULCER_INDEX',
'DPO',
'KST',
'KST_SIG',
'MACD',
'MACD_DIFF',
'MACD_SIGNAL',
'STC',
'TRIX',
'CUMULATIVE_RETURN',
'DAILY_LOG_RETURN',
'DAILY_RETURN',
'PVO',
'PVO_HIST',
'PVO_SIGNAL',
'CCI',
'STOCH',
'STOCH_SIGNAL',
'ULTIMATE_OSCILLATOR',
'WILLIAMS_R',
'AVERAGE_TRUE_RANGE',
# 'DONCHIAN_CHANNEL_HBAND',
# 'DONCHIAN_CHANNEL_LBAND',
# 'DONCHIAN_CHANNEL_MBAND',
'DONCHIAN_CHANNEL_PBAND',
'DONCHIAN_CHANNEL_WBAND',
# 'KELTNER_CHANNEL_HBAND',
'KELTNER_CHANNEL_HBAND_INDICATOR',
# 'KELTNER_CHANNEL_LBAND',
'KELTNER_CHANNEL_LBAND_INDICATOR',
# 'KELTNER_CHANNEL_MBAND',
'KELTNER_CHANNEL_PBAND',
'KELTNER_CHANNEL_WBAND',
'ADX',
'ADX_NEG',
'ADX_POS',
'PSAR_DOWN',
'PSAR_DOWN_INDICATOR',
'PSAR_UP',
'PSAR_UP_INDICATOR',
'VORTEX_INDICATOR_NEG',
'VORTEX_INDICATOR_POS',
'EASE_OF_MOVEMENT',
'SMA_EASE_OF_MOVEMENT',
'ACC_DIST_INDEX',
'CHAIKIN_MONEY_FLOW',
'MONEY_FLOW_INDEX',
# 'VOLUME_WEIGHTED_AVERAGE_PRICE',
'FORCE_INDEX',
# 'NEGATIVE_VOLUME_INDEX',
'ON_BALANCE_VOLUME',
'VOLUME_PRICE_TREND',
'AWESOME_OSCILLATOR',
# 'ICHIMOKU_A',
# 'ICHIMOKU_B',
# 'ICHIMOKU_BASE_LINE',
# 'ICHIMOKU_CONVERSION_LINE',
'MASS_INDEX',
'AROON_DOWN',
'AROON_INDICATOR',
'AROON_UP',
# 'EMA_INDICATOR_012',
# 'EMA_INDICATOR_024',
# 'EMA_INDICATOR_048',
# 'EMA_INDICATOR_072',
# 'EMA_INDICATOR_096',
# 'EMA_INDICATOR_120',
# 'EMA_INDICATOR_144',
# 'EMA_INDICATOR_168',
# 'EMA_INDICATOR_192',
# 'SMA_INDICATOR_012',
# 'SMA_INDICATOR_024',
# 'SMA_INDICATOR_048',
# 'SMA_INDICATOR_072',
# 'SMA_INDICATOR_096',
# 'SMA_INDICATOR_120',
# 'SMA_INDICATOR_144',
# 'SMA_INDICATOR_168',
# 'SMA_INDICATOR_192',
'Day_sin',
'Day_cos',
'One_Year_sin',
'One_Year_cos',
'Two_Year_sin',
'Two_Year_cos',
'Three_Year_sin',
'Three_Year_cos',
'Four_Year_sin',
'Four_Year_cos',
'Five_Year_sin',
'Five_Year_cos',
# 'buy_1',
# 'buy_1.25',
# 'buy_1.5',
# 'buy_1.75',
# 'buy_2',
# 'buy_4',
# 'buy_6',
# 'sell_1',
# 'sell_1.25',
# 'sell_1.5',
# 'sell_1.75',
# 'sell_2',
# 'sell_4',
# 'sell_6',
'BOLLINGER_HBAND_C',
'BOLLINGER_LBAND_C',
'BOLLINGER_MAVG_C',
'DONCHIAN_CHANNEL_HBAND_C',
'DONCHIAN_CHANNEL_LBAND_C',
'DONCHIAN_CHANNEL_MBAND_C',
'EMA_INDICATOR_012_C',
'EMA_INDICATOR_024_C',
'EMA_INDICATOR_048_C',
'EMA_INDICATOR_072_C',
'EMA_INDICATOR_096_C',
'EMA_INDICATOR_120_C',
'EMA_INDICATOR_144_C',
'EMA_INDICATOR_168_C',
'EMA_INDICATOR_192_C',
'SMA_INDICATOR_012_C',
'SMA_INDICATOR_024_C',
'SMA_INDICATOR_048_C',
'SMA_INDICATOR_072_C',
'SMA_INDICATOR_096_C',
'SMA_INDICATOR_120_C',
'SMA_INDICATOR_144_C',
'SMA_INDICATOR_168_C',
'SMA_INDICATOR_192_C',
# 'Open_C',
# 'High_C',
# 'Low_C',
'ICHIMOKU_A_C',
'ICHIMOKU_B_C',
'ICHIMOKU_BASE_LINE_C',
'ICHIMOKU_CONVERSION_LINE_C',
'KELTNER_CHANNEL_HBAND_C',
'KELTNER_CHANNEL_LBAND_C',
'KELTNER_CHANNEL_MBAND_C',
'NEGATIVE_VOLUME_INDEX_C',
'VOLUME_WEIGHTED_AVERAGE_PRICE_C',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_024',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_048',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_072',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_012_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_012_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_048',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_072',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_024_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_024_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_072',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_048_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_048_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_096',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_072_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_072_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_120',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_096_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_096_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_120_TO_EMA_INDICATOR_144',
'EMA_INDICATOR_120_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_120_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_120_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_144_TO_EMA_INDICATOR_168',
'EMA_INDICATOR_144_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_144_TO_SMA_INDICATOR_192',
'EMA_INDICATOR_168_TO_EMA_INDICATOR_192',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_168_TO_SMA_INDICATOR_192',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_012',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_024',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_048',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_072',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_096',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_120',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_144',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_168',
# 'EMA_INDICATOR_192_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_024',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_048',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_072',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_012_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_048',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_072',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_024_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_072',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_048_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_096',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_072_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_120',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_096_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_120_TO_SMA_INDICATOR_144',
# 'SMA_INDICATOR_120_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_120_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_144_TO_SMA_INDICATOR_168',
# 'SMA_INDICATOR_144_TO_SMA_INDICATOR_192',
# 'SMA_INDICATOR_168_TO_SMA_INDICATOR_192',
# 'DateTime',
]

print(f"{len(buy_features)=},{len(sell_features)=}")

In [8]:
buy_column = "buy_1"
sell_column = "sell_1"


X_train_sell = train_df[sell_features_names]
X_test_sell = test_df[sell_features_names]

y_train_sell = pd.concat([train_df[sell_column],train_df[sell_column].subtract(1).abs()] , axis = 1).to_numpy()
y_test_sell = pd.concat([test_df[sell_column],test_df[sell_column].subtract(1).abs()] , axis = 1).to_numpy()


X_train_buy = train_df[buy_features_names]
X_test_buy = test_df[buy_features_names]

y_train_buy = pd.concat([train_df[buy_column],train_df[buy_column].subtract(1).abs()] , axis = 1).to_numpy()
y_test_buy = pd.concat([test_df[buy_column],test_df[buy_column].subtract(1).abs()] , axis = 1).to_numpy()

print(f"{X_train_sell.shape=}\n{X_test_sell.shape=}\n{y_train_buy.shape=}\n{y_test_buy.shape=}")


X_train_sell.shape=(7029, 240)
X_test_sell.shape=(3024, 240)
y_train_buy.shape=(7029, 2)
y_test_buy.shape=(3024, 2)


In [14]:
train_df.columns.tolist()

['Open',
 'High',
 'Low',
 'Close',
 'TickVol',
 'Spread',
 'PPO',
 'HIST',
 'PPO_SIGNAL',
 'ROC',
 'RSI',
 'STOCHRSI',
 'STOCHRSI_D',
 'STOCHRSI_K',
 'TSI',
 'BOLLINGER_HBAND',
 'BOLLINGER_HBAND_INDICATOR',
 'BOLLINGER_LBAND',
 'BOLLINGER_LBAND_INDICATOR',
 'BOLLINGER_MAVG',
 'BOLLINGER_PBAND',
 'BOLLINGER_WBAND',
 'ULCER_INDEX',
 'DPO',
 'KST',
 'KST_SIG',
 'MACD',
 'MACD_DIFF',
 'MACD_SIGNAL',
 'STC',
 'TRIX',
 'CUMULATIVE_RETURN',
 'DAILY_LOG_RETURN',
 'DAILY_RETURN',
 'PVO',
 'PVO_HIST',
 'PVO_SIGNAL',
 'CCI',
 'STOCH',
 'STOCH_SIGNAL',
 'ULTIMATE_OSCILLATOR',
 'WILLIAMS_R',
 'AVERAGE_TRUE_RANGE',
 'DONCHIAN_CHANNEL_HBAND',
 'DONCHIAN_CHANNEL_LBAND',
 'DONCHIAN_CHANNEL_MBAND',
 'DONCHIAN_CHANNEL_PBAND',
 'DONCHIAN_CHANNEL_WBAND',
 'KELTNER_CHANNEL_HBAND',
 'KELTNER_CHANNEL_HBAND_INDICATOR',
 'KELTNER_CHANNEL_LBAND',
 'KELTNER_CHANNEL_LBAND_INDICATOR',
 'KELTNER_CHANNEL_MBAND',
 'KELTNER_CHANNEL_PBAND',
 'KELTNER_CHANNEL_WBAND',
 'ADX',
 'ADX_NEG',
 'ADX_POS',
 'PSAR_DOWN',
 'PSAR_

In [9]:
#MODEL
from xgboost import XGBRFClassifier
from sklearn.metrics import log_loss , precision_score

size = 2000
max_depth=12
# learning_rate = 1
# max_bin=512

buy_model = XGBRFClassifier(
                        n_estimators=size,
                        max_depth=max_depth,
                        max_leaves=0,
                        # max_bin=max_bin,
                        subsample=0.9,
                        colsample_bynode=0.2,
                        grow_policy='depthwise',
                        # learning_rate=learning_rate,
                        verbosity = 1,
                        booster='gbtree',
                        tree_method = "gpu_hist",
                        n_jobs=0,
                        gamma=0,
                        min_child_weight=1,
                        sampling_method='gradient_based',
                        colsample_bytree=1,
                        missing=np.nan,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        base_score=0.5,
                        random_state=23,
                        num_parallel_tree=size*5,
                        gpu_id=0,
                        predictor='auto',
                        eval_metric = precision_score,
                        max_delta_step=0,
                        # num_class = 2,
                        # objective='multi:softprob',
                        )

sell_model = XGBRFClassifier(
                        n_estimators=size,
                        max_depth=max_depth,
                        max_leaves=0,
                        # max_bin=max_bin,
                        subsample=0.9,
                        colsample_bynode=0.2,
                        grow_policy='depthwise',
                        # learning_rate=learning_rate,
                        verbosity = 1,
                        booster='gbtree',
                        tree_method = "gpu_hist",
                        n_jobs=0,
                        gamma=0,
                        min_child_weight=1,
                        sampling_method='gradient_based',
                        colsample_bytree=1,
                        missing=np.nan,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        base_score=0.5,
                        random_state=23,
                        num_parallel_tree=size*5,
                        gpu_id=0,
                        predictor='auto',
                        eval_metric = precision_score,
                        max_delta_step=0,
                        # num_class = 2,
                        # objective='multi:softprob',
                        )

In [10]:
buy_model.fit(
            X_train_buy,
            y_train_buy,
            # eval_set = [(X_test , y_test)]
        )



XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=<function precision_score at 0x000001C4BD4A2E50>,
                gamma=0, gpu_id=0, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=12,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=2000, n_jobs=0,
                num_parallel_tree=10000, objective='binary:logistic',
                predictor='auto', random_state=23, reg_alpha=0,
                sampling_method='gradient_based', ...)

In [11]:
sell_model.fit(
            X_train_sell,
            y_train_sell,
            # eval_set = [(X_test , y_test)]
        )


XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=<function precision_score at 0x000001C4BD4A2E50>,
                gamma=0, gpu_id=0, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=12,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=2000, n_jobs=0,
                num_parallel_tree=10000, objective='binary:logistic',
                predictor='auto', random_state=23, reg_alpha=0,
                sampling_method='gradient_based', ...)

In [12]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

p_train_buy = buy_model.predict(X_train_buy)
p_test_buy = buy_model.predict(X_test_buy)

rep_train = classification_report(y_train_buy,p_train_buy)
rep_test = classification_report(y_test_buy,p_test_buy)

print(rep_train)
print(rep_test)


              precision    recall  f1-score   support

           0       1.00      0.80      0.89       556
           1       0.98      1.00      0.99      6473

   micro avg       0.98      0.98      0.98      7029
   macro avg       0.99      0.90      0.94      7029
weighted avg       0.98      0.98      0.98      7029
 samples avg       0.98      0.98      0.98      7029

              precision    recall  f1-score   support

           0       0.11      0.20      0.14       177
           1       0.95      0.90      0.92      2847

   micro avg       0.86      0.86      0.86      3024
   macro avg       0.53      0.55      0.53      3024
weighted avg       0.90      0.86      0.88      3024
 samples avg       0.86      0.86      0.86      3024



c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

p_train_sell = sell_model.predict(X_train_sell)
p_test_sell = sell_model.predict(X_test_sell)

rep_train = classification_report(y_train_sell,p_train_sell)
rep_test = classification_report(y_test_sell,p_test_sell)

print(rep_train)
print(rep_test)


              precision    recall  f1-score   support

           0       1.00      0.92      0.96       878
           1       0.99      1.00      0.99      6151

   micro avg       0.99      0.99      0.99      7029
   macro avg       0.99      0.96      0.97      7029
weighted avg       0.99      0.99      0.99      7029
 samples avg       0.99      0.99      0.99      7029

              precision    recall  f1-score   support

           0       0.62      0.09      0.15       512
           1       0.84      0.99      0.91      2512

   micro avg       0.84      0.83      0.84      3024
   macro avg       0.73      0.54      0.53      3024
weighted avg       0.80      0.83      0.78      3024
 samples avg       0.83      0.83      0.83      3024



c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from xgboost import plot_importance
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["figure.figsize"]=(20,40)

plot_importance(sell_model,max_num_features= 100)

In [ ]:
a = zip(sell_model.feature_names_in_,sell_model.feature_importances_)

a = dict(a)

b = dict(sorted(a.items(), key=lambda item: item[1] , reverse=True))
b

In [ ]:
sell_model.get_params()

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

lgbm_model = LGBMClassifier(
    n_estimators = 500,
    objective = "multiclass",
    random_state = 23,
    max_depth=12,
    learning_rate = .01,
    num_leaves = 31,
)

lgbm_model.fit(
    X = X_train,
    y = y_train,
    eval_set=(X_test,y_test)
    
)

p_train_lgbm = lgbm_model.predict(X_train)
p_test_lgbm = lgbm_model.predict(X_test)

acc_train_lgbm = accuracy_score(p_train_lgbm,y_train)
acc_test_lgbm = accuracy_score(p_test_lgbm,y_test)

rep_train_lgbm = classification_report(y_train,p_train_lgbm)
rep_test_lgbm = classification_report(y_test,p_test_lgbm)

con_train_lgbm = confusion_matrix(y_train,p_train_lgbm)
con_test_lgbm = confusion_matrix(y_test,p_test_lgbm)

print(rep_train_lgbm)
print(rep_test_lgbm)


In [24]:
limit = -10_000
df = test_df[limit:]
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

buys = []
for i in range(p_test_buy.shape[0]):
    buys.append(p_test_buy[i][0])
 
sells = []
for i in range(p_test_sell.shape[0]):
    sells.append(p_test_sell[i][0])   
    
df["Buy"] = buys[limit:]
df["Sell"]= sells[limit:]

In [25]:
df["Sell"].sum()

1455.0

In [34]:
from backtesting import Strategy

def buy():
    return df.Buy

def sell():
    return df.Sell

def Class():
    return df.Class

def vwap_signal():
    return df.TotalSignal


class MyStrategy(Strategy):


    def init(self):
        super().init()
        self.initsize = .01
        self.mysize = self.initsize
        self.buy_signal = self.I(buy)
        self.sell_signal = self.I(sell)


    def next(self):
        super().next()
        
        sl_to_atr = 4 * 1
        tp_to_atr = 1 * sl_to_atr
        
        # if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
        #     if self.mysize < 128_000:
        #         self.mysize = int(self.mysize * 1.1)
        #         if self.mysize > 128_000:
        #             self.mysize = 128_000
        #     else:
        #         self.c += 1

        # elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
        #     if self.c > 0:
        #         self.c -= 1
        #     else:
        #         self.mysize = self.initsize
        
        if self.buy_signal == 1 and 0 and len(self.trades) < 5:
            
            sl1 = self.data.Close[-1] - (sl_to_atr * max(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #- self.data.Spread[-1]#*1e-4
            tp1 = self.data.Close[-1] + (tp_to_atr * max(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #+ self.data.Spread[-1]#*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if self.sell_signal == 1 and 1 and len(self.trades) < 5:
            
            sl1=self.data.Close[-1] + (sl_to_atr * max(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #+ self.data.Spread[-1]#*1e-4
            tp1=self.data.Close[-1] - (tp_to_atr * max(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #- self.data.Spread[-1]#*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            


from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=10_000, margin=1/100, commission=.0002,hedging=True)
stat=bt.run()
# stat.to_csv(f"{path}/stat.csv")
stat

Start                     2022-02-02 03:15:00
End                       2022-07-15 21:00:00
Duration                    163 days 17:45:00
Exposure Time [%]                       45.53
Equity Final [$]                 17701.569072
Equity Peak [$]                  18676.621827
Return [%]                          77.015691
Buy & Hold Return [%]              -15.490334
Return (Ann.) [%]                  238.572938
Volatility (Ann.) [%]              192.245816
Sharpe Ratio                         1.240979
Sortino Ratio                        7.055809
Calmar Ratio                        10.481819
Max. Drawdown [%]                  -22.760643
Avg. Drawdown [%]                   -2.079882
Max. Drawdown Duration       61 days 19:15:00
Avg. Drawdown Duration        1 days 16:33:00
# Trades                                  339
Win Rate [%]                        62.536873
Best Trade [%]                       2.752186
Worst Trade [%]                     -2.738408
Avg. Trade [%]                    

In [29]:
bt.plot(results=None,
        filename=None,
        plot_width=None,
        plot_equity=True,
        plot_return=False,
        plot_pl=True,
        plot_volume=False,
        plot_drawdown=False,
        smooth_equity=False,
        relative_equity=True,
        superimpose=True,
        resample=True,
        reverse_indicators=False,
        show_legend=True,
        open_browser=True)

Row(id='1484', ...)

In [41]:
from xgboost import plot_importance ,plot_tree, to_graphviz , Booster
import pathlib



def save():
    buy_model.save_model(f"./tmp/buy_model_{name}.json")
    sell_model.save_model(f"./tmp/sell_model_{name}.json")
    
    # check if path is available
    path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}_{tp_to_sl:02}'
    pathlib.Path(path).mkdir(parents=True,exist_ok=True)

    #SAVE

    with open(f"{path}/train{tp_to_sl:02}_report.txt","w") as f:
        f.write(rep_train)

    with open(f"{path}/test{tp_to_sl:02}_report.txt","w") as f:
        f.write(rep_test) 
        
    model.save_model(f"{path}/model{tp_to_sl:02}.json")

    train_df.mean().to_pickle(f"{path}/mean.pickle")
    train_df.std().to_pickle(f"{path}/std.pickle")

    train_df.to_pickle(f"{path}/train_df.pickle")
    test_df.to_pickle(f"{path}/test_df.pickle")
        
    #LOAD    

    # model = XGBRFClassifier()
    # model.load_model(f"{path}/model.json")


    #Drow model
    graph = to_graphviz(model)
    graph.render(filename=f"{path}/model")

c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\xgboost\sklearn.py:722: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


'D:\\dev\\workspace\\vscode\\python\\bos\\model\\models\\XGBOOST\\SP500m_H1_202009231400_202207082200_slatr_05_01\\model.pdf'

In [44]:
1/(1+np.exp(-1*0.000999998767))

0.5002499996709168

In [ ]:
# Normalizing with z-score https://developers.google.com/machine-learning/data-prep/transform/normalization

# train_df.mean().to_csv(f"{data_settings.base_path}/data/csv/train_mean.csv")
# train_df.std().to_csv(f"{data_settings.base_path}/data/csv/train_std.csv")
# train_df = (train_df - train_df.mean())/train_df.std()
# test_df = (test_df - train_df.mean())/train_df.std()

# Corrilation of buy and sell and do nothing with other rows of dataframes
# pd.options.display.max_rows =1000

# train_df.corr()["buy_4"]
# train_df.corr()["sell_4"]
# train_df.corr()["do_nothing"]

# import matplotlib.pyplot as plt

# for column in df.columns:
    
#     hist = df[column].hist(bins=100)
#     plt.savefig(f"./tmp/{column}_normalized.png")
#     hist.clear()


In [ ]:
# from lightgbm import LGBMClassifier

# lgbm_model = LGBMClassifier(
#     n_estimators = 1_000,
#     objective = "multiclass",
#     random_state = 23,
#     learning_rate = .01,
#     num_leaves = 128
# )

# lgbm_model.fit(
#     X = X_train,
#     y = y_train
    
# )
# from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

# p_train_lgbm = lgbm_model.predict(X_train)
# p_test_lgbm = lgbm_model.predict(X_test)

# acc_train_lgbm = accuracy_score(p_train,y_train)
# acc_test_lgbm = accuracy_score(p_test,y_test)

# rep_train_lgbm = classification_report(y_train,p_train)
# rep_test_lgbm = classification_report(y_test,p_test)

# con_train_lgbm = confusion_matrix(y_train,p_train)
# con_test_lgbm = confusion_matrix(y_test,p_test)

# print(rep_train_lgbm)
# print(rep_test_lgbm)


In [ ]:
buy_features_names = [
    'TickVol',
    'Spread',
    'PPO',
    'HIST',
    'PPO_SIGNAL',
    'RSI',
    'STOCHRSI',
    'STOCHRSI_D',
    'STOCHRSI_K',
    'TSI',
    'BOLLINGER_HBAND_INDICATOR',
    'BOLLINGER_LBAND_INDICATOR',
    'BOLLINGER_PBAND',
    'BOLLINGER_WBAND',
    'DPO',
    'MACD',
    'MACD_DIFF',
    'MACD_SIGNAL',
    'STC',
    'TRIX',
    'PVO',
    'PVO_HIST',
    'PVO_SIGNAL',
    'CCI',
    'STOCH',
    'STOCH_SIGNAL',
    'ULTIMATE_OSCILLATOR',
    'WILLIAMS_R',
    'AVERAGE_TRUE_RANGE',
    'DONCHIAN_CHANNEL_PBAND',
    'DONCHIAN_CHANNEL_WBAND',
    'KELTNER_CHANNEL_HBAND_INDICATOR',
    'KELTNER_CHANNEL_LBAND_INDICATOR',
    'KELTNER_CHANNEL_PBAND',
    'KELTNER_CHANNEL_WBAND',
    'ADX',
    'ADX_NEG',
    'ADX_POS',
    'VORTEX_INDICATOR_NEG',
    'VORTEX_INDICATOR_POS',
    'EASE_OF_MOVEMENT',
    'SMA_EASE_OF_MOVEMENT',
    'ACC_DIST_INDEX',
    'CHAIKIN_MONEY_FLOW',
    'MONEY_FLOW_INDEX',
    'VOLUME_WEIGHTED_AVERAGE_PRICE',
    'FORCE_INDEX',
    'ON_BALANCE_VOLUME',
    'AWESOME_OSCILLATOR',
    'MASS_INDEX',
    'AROON_DOWN',
    'AROON_INDICATOR',
    'AROON_UP',
    'Day_sin',
    'Day_cos',
    'One_Year_sin',
    'One_Year_cos',
    'Two_Year_sin',
    'Two_Year_cos',
    'Three_Year_sin',
    'Three_Year_cos',
    'Four_Year_sin',
    'Four_Year_cos',
    'Five_Year_sin',
    'Five_Year_cos',
    'BOLLINGER_HBAND_C',
    'BOLLINGER_LBAND_C',
    'BOLLINGER_MAVG_C',
    'DONCHIAN_CHANNEL_HBAND_C',
    'DONCHIAN_CHANNEL_LBAND_C',
    'DONCHIAN_CHANNEL_MBAND_C',
    'EMA_INDICATOR_012_C',
    'EMA_INDICATOR_024_C',
    'EMA_INDICATOR_048_C',
    'EMA_INDICATOR_072_C',
    'EMA_INDICATOR_096_C',
    'EMA_INDICATOR_120_C',
    'EMA_INDICATOR_144_C',
    'EMA_INDICATOR_168_C',
    'EMA_INDICATOR_192_C',
    'SMA_INDICATOR_012_C',
    'SMA_INDICATOR_024_C',
    'SMA_INDICATOR_048_C',
    'SMA_INDICATOR_072_C',
    'SMA_INDICATOR_096_C',
    'SMA_INDICATOR_120_C',
    'SMA_INDICATOR_144_C',
    'SMA_INDICATOR_168_C',
    'SMA_INDICATOR_192_C',
    'ICHIMOKU_A_C',
    'ICHIMOKU_B_C',
    'ICHIMOKU_BASE_LINE_C',
    'ICHIMOKU_CONVERSION_LINE_C',
    'KELTNER_CHANNEL_HBAND_C',
    'KELTNER_CHANNEL_LBAND_C',
    'KELTNER_CHANNEL_MBAND_C',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_024',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_048',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_072',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_096',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_120',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_144',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_012_TO_EMA_INDICATOR_192',
    # 'EMA_INDICATOR_012_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_012_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_048',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_072',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_096',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_120',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_144',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_024_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_012',
    # 'EMA_INDICATOR_024_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_024_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_048_TO_EMA_INDICATOR_072',
    'EMA_INDICATOR_048_TO_EMA_INDICATOR_096',
    'EMA_INDICATOR_048_TO_EMA_INDICATOR_120',
    'EMA_INDICATOR_048_TO_EMA_INDICATOR_144',
    'EMA_INDICATOR_048_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_048_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_024',
    # 'EMA_INDICATOR_048_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_048_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_072_TO_EMA_INDICATOR_096',
    'EMA_INDICATOR_072_TO_EMA_INDICATOR_120',
    'EMA_INDICATOR_072_TO_EMA_INDICATOR_144',
    'EMA_INDICATOR_072_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_072_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_048',
    # 'EMA_INDICATOR_072_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_072_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_096_TO_EMA_INDICATOR_120',
    'EMA_INDICATOR_096_TO_EMA_INDICATOR_144',
    'EMA_INDICATOR_096_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_096_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_072',
    # 'EMA_INDICATOR_096_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_096_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_120_TO_EMA_INDICATOR_144',
    'EMA_INDICATOR_120_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_120_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_096',
    # 'EMA_INDICATOR_120_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_120_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_144_TO_EMA_INDICATOR_168',
    'EMA_INDICATOR_144_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_120',
    # 'EMA_INDICATOR_144_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_144_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_168_TO_EMA_INDICATOR_192',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_144',
    # 'EMA_INDICATOR_168_TO_SMA_INDICATOR_168',
    'EMA_INDICATOR_168_TO_SMA_INDICATOR_192',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_012',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_024',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_048',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_072',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_096',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_120',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_144',
    'EMA_INDICATOR_192_TO_SMA_INDICATOR_168',
    # 'EMA_INDICATOR_192_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_024',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_048',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_072',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_096',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_120',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_144',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_012_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_048',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_072',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_096',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_120',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_144',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_024_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_048_TO_SMA_INDICATOR_072',
    'SMA_INDICATOR_048_TO_SMA_INDICATOR_096',
    'SMA_INDICATOR_048_TO_SMA_INDICATOR_120',
    'SMA_INDICATOR_048_TO_SMA_INDICATOR_144',
    'SMA_INDICATOR_048_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_048_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_072_TO_SMA_INDICATOR_096',
    'SMA_INDICATOR_072_TO_SMA_INDICATOR_120',
    'SMA_INDICATOR_072_TO_SMA_INDICATOR_144',
    'SMA_INDICATOR_072_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_072_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_096_TO_SMA_INDICATOR_120',
    'SMA_INDICATOR_096_TO_SMA_INDICATOR_144',
    'SMA_INDICATOR_096_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_096_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_120_TO_SMA_INDICATOR_144',
    'SMA_INDICATOR_120_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_120_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_144_TO_SMA_INDICATOR_168',
    'SMA_INDICATOR_144_TO_SMA_INDICATOR_192',
    'SMA_INDICATOR_168_TO_SMA_INDICATOR_192',
]
